### Limpieza de los datos

In [1]:
import pandas as pd
import glob
import numpy as np

In [2]:
# juntamos todos los accidentes en uno solo archivo.
# get data file names
path =r'../data/atus_anual_1997_2019/conjunto_de_datos/'
filenames = glob.glob(path + "/*.csv")
dfs = []
# datos que nos interesan. 
col_analisis=['id_entidad', 'id_municipio', 'anio', 'mes', 'id_hora',
       'id_minuto', 'id_dia', 'diasemana', 'tipaccid',
        'causaacci', 'caparod', 'sexo', 'aliento',
       'cinturon', 'id_edad', "clasacc"]

for filename in filenames:
    df_year=pd.read_csv(filename, index_col=False)
    df_year.columns = [item.lower() for item in df_year.columns.to_list()]
    dfs.append(df_year[col_analisis])
    print(filename)


# Concatenate all data into one DataFrame
big_frame = pd.concat(dfs, ignore_index=True)
big_frame.to_csv("../data/atus_anual_1997_2019.csv", index=False)

../data/atus_anual_1997_2019/conjunto_de_datos/atus_anual_2017.csv
../data/atus_anual_1997_2019/conjunto_de_datos/atus_anual_2019.csv
../data/atus_anual_1997_2019/conjunto_de_datos/atus_anual_2003.csv
../data/atus_anual_1997_2019/conjunto_de_datos/atus_anual_2012.csv
../data/atus_anual_1997_2019/conjunto_de_datos/atus_anual_2001.csv
../data/atus_anual_1997_2019/conjunto_de_datos/atus_anual_2010.csv
../data/atus_anual_1997_2019/conjunto_de_datos/atus_anual_2004.csv
../data/atus_anual_1997_2019/conjunto_de_datos/atus_anual_2005.csv
../data/atus_anual_1997_2019/conjunto_de_datos/atus_anual_1999.csv
../data/atus_anual_1997_2019/conjunto_de_datos/atus_anual_2002.csv
../data/atus_anual_1997_2019/conjunto_de_datos/atus_anual_2018.csv
../data/atus_anual_1997_2019/conjunto_de_datos/atus_anual_2013.csv
../data/atus_anual_1997_2019/conjunto_de_datos/atus_anual_2016.csv
../data/atus_anual_1997_2019/conjunto_de_datos/atus_anual_2008.csv
../data/atus_anual_1997_2019/conjunto_de_datos/atus_anual_2009

In [3]:
accidentes_mexico = pd.read_csv("../data/raw/atus_anual_1997_2019.csv")

In [4]:
accidentes_mexico.shape


(8876380, 16)

In [5]:
accidentes_mexico.head()

,id_entidad,id_municipio,anio,mes,id_hora,id_minuto,id_dia,diasemana,tipaccid,causaacci,caparod,sexo,aliento,cinturon,id_edad,clasacc
0,1,1.0,2017,1,18,30,11,Miércoles,Colisión con vehículo automotor,Conductor,Pavimentada,Hombre,No,Se ignora,30,Sólo daños
1,1,1.0,2017,1,1,50,9,Lunes,Colisión con motocicleta,Conductor,Pavimentada,Hombre,No,Se ignora,30,No fatal
2,1,1.0,2017,1,8,30,12,Jueves,Colisión con objeto fijo,Conductor,Pavimentada,Hombre,No,Se ignora,22,No fatal
3,1,1.0,2017,1,17,40,9,Lunes,Colisión con motocicleta,Conductor,Pavimentada,Mujer,No,Se ignora,73,Sólo daños
4,1,1.0,2017,1,21,10,11,Miércoles,Colisión con objeto fijo,Conductor,Pavimentada,Hombre,No,Se ignora,33,No fatal


In [6]:
accidentes_mexico.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8876380 entries, 0 to 8876379
Data columns (total 16 columns):
 #   Column        Dtype  
---  ------        -----  
 0   id_entidad    int64  
 1   id_municipio  float64
 2   anio          int64  
 3   mes           int64  
 4   id_hora       int64  
 5   id_minuto     int64  
 6   id_dia        int64  
 7   diasemana     object 
 8   tipaccid      object 
 9   causaacci     object 
 10  caparod       object 
 11  sexo          object 
 12  aliento       object 
 13  cinturon      object 
 14  id_edad       int64  
 15  clasacc       object 
dtypes: float64(1), int64(7), object(8)
memory usage: 1.1+ GB


In [7]:
print(accidentes_mexico.id_entidad.unique())

# concatenamos con los catalogos
cat_estado=pd.read_csv("../data/raw/atus_anual_1997_2019/catalogos/tc_entidad.csv", index_col=False)
cat_estado.columns = [item.lower() for item in cat_estado.columns.to_list()]

print(accidentes_mexico.shape)
accidentes_mexico=pd.merge(accidentes_mexico, cat_estado, on="id_entidad", how="left")
print(accidentes_mexico.shape)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32]
(8876380, 16)
(8876380, 17)


In [8]:
accidentes_mexico.head()

,id_entidad,id_municipio,anio,mes,id_hora,id_minuto,id_dia,diasemana,tipaccid,causaacci,caparod,sexo,aliento,cinturon,id_edad,clasacc,nom_entidad
0,1,1.0,2017,1,18,30,11,Miércoles,Colisión con vehículo automotor,Conductor,Pavimentada,Hombre,No,Se ignora,30,Sólo daños,Aguascalientes
1,1,1.0,2017,1,1,50,9,Lunes,Colisión con motocicleta,Conductor,Pavimentada,Hombre,No,Se ignora,30,No fatal,Aguascalientes
2,1,1.0,2017,1,8,30,12,Jueves,Colisión con objeto fijo,Conductor,Pavimentada,Hombre,No,Se ignora,22,No fatal,Aguascalientes
3,1,1.0,2017,1,17,40,9,Lunes,Colisión con motocicleta,Conductor,Pavimentada,Mujer,No,Se ignora,73,Sólo daños,Aguascalientes
4,1,1.0,2017,1,21,10,11,Miércoles,Colisión con objeto fijo,Conductor,Pavimentada,Hombre,No,Se ignora,33,No fatal,Aguascalientes


In [11]:
print(len(accidentes_mexico.id_municipio.unique()))

# concatenamos con los catalogos
cat_muni=pd.read_csv("../data/raw/atus_anual_1997_2019/catalogos/tc_municipio.csv", index_col=False)
cat_muni.columns = [item.lower() for item in cat_muni.columns.to_list()]

print(accidentes_mexico.shape)
accidentes_mexico=pd.merge(accidentes_mexico, cat_muni, on=["id_entidad","id_municipio"], how="left")
print(accidentes_mexico.shape)

571


In [14]:
accidentes_mexico.head()

,id_entidad,id_municipio,anio,mes,id_hora,id_minuto,id_dia,diasemana,tipaccid,causaacci,caparod,sexo,aliento,cinturon,id_edad,clasacc,nom_entidad,nom_municipio
0,1,1.0,2017,1,18,30,11,Miércoles,Colisión con vehículo automotor,Conductor,Pavimentada,Hombre,No,Se ignora,30,Sólo daños,Aguascalientes,Aguascalientes
1,1,1.0,2017,1,1,50,9,Lunes,Colisión con motocicleta,Conductor,Pavimentada,Hombre,No,Se ignora,30,No fatal,Aguascalientes,Aguascalientes
2,1,1.0,2017,1,8,30,12,Jueves,Colisión con objeto fijo,Conductor,Pavimentada,Hombre,No,Se ignora,22,No fatal,Aguascalientes,Aguascalientes
3,1,1.0,2017,1,17,40,9,Lunes,Colisión con motocicleta,Conductor,Pavimentada,Mujer,No,Se ignora,73,Sólo daños,Aguascalientes,Aguascalientes
4,1,1.0,2017,1,21,10,11,Miércoles,Colisión con objeto fijo,Conductor,Pavimentada,Hombre,No,Se ignora,33,No fatal,Aguascalientes,Aguascalientes


In [15]:
# imprimimos los años
accidentes_mexico.anio.unique()

array([2017, 2019, 2003, 2012, 2001, 2010, 2004, 2005, 1999, 2002, 2018,
       2013, 2016, 2008, 2009, 2014, 2000, 2007, 2011, 1998, 2015, 1997,
       2006])

In [16]:
accidentes_mexico.mes.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [17]:
accidentes_mexico[accidentes_mexico.id_dia==0].head()

,id_entidad,id_municipio,anio,mes,id_hora,id_minuto,id_dia,diasemana,tipaccid,causaacci,caparod,sexo,aliento,cinturon,id_edad,clasacc,nom_entidad,nom_municipio
3934119,1,8.0,2018,1,0,0,0,Certificado cero,Certificado cero,Certificado cero,Certificado cero,Certificado cero,Certificado cero,Certificado cero,0,Certificado cero,Aguascalientes,San José de Gracia
3934425,1,8.0,2018,2,0,0,0,Certificado cero,Certificado cero,Certificado cero,Certificado cero,Certificado cero,Certificado cero,Certificado cero,0,Certificado cero,Aguascalientes,San José de Gracia
3935055,1,4.0,2018,4,0,0,0,Certificado cero,Certificado cero,Certificado cero,Certificado cero,Certificado cero,Certificado cero,Certificado cero,0,Certificado cero,Aguascalientes,Cosío
3935120,1,10.0,2018,4,0,0,0,Certificado cero,Certificado cero,Certificado cero,Certificado cero,Certificado cero,Certificado cero,Certificado cero,0,Certificado cero,Aguascalientes,El Llano
3935376,1,4.0,2018,5,0,0,0,Certificado cero,Certificado cero,Certificado cero,Certificado cero,Certificado cero,Certificado cero,Certificado cero,0,Certificado cero,Aguascalientes,Cosío


In [18]:
# analicemos la existencia de ceros en los días
accidentes_dia_cero = accidentes_mexico[accidentes_mexico.id_dia==0]

In [19]:
accidentes_dia_cero.anio.unique() # todos estan en el año 2018

array([2018])

In [20]:
accidentes_dia_cero.nom_entidad.value_counts() # la mayoria estan en Oaxaca.

Oaxaca                             6361
Puebla                             2207
Veracruz de Ignacio de la Llave    1584
Chiapas                            1128
Hidalgo                             865
San Luis Potosi                     434
Sonora                              431
Tlaxcala                            348
Tamaulipas                          265
Michoacan de Ocampo                 251
Yucatan                             227
Mexico                              211
Nuevo Leon                          164
Zacatecas                           164
Chihuahua                           150
Coahuila de Zaragoza                137
Nayarit                              73
Durango                              59
Guanajuato                           55
Tabasco                              25
Queretaro                            21
Aguascalientes                       19
Morelos                              11
Campeche                             11
Sinaloa                               7


In [21]:
for i in accidentes_dia_cero.columns[4:16]:
    print("La columna",i,"tiene los valores únicos:", accidentes_dia_cero[i].unique())

La columna id_hora tiene los valores únicos: [0]
La columna id_minuto tiene los valores únicos: [0]
La columna id_dia tiene los valores únicos: [0]
La columna diasemana tiene los valores únicos: ['Certificado cero']
La columna tipaccid tiene los valores únicos: ['Certificado cero']
La columna causaacci tiene los valores únicos: ['Certificado cero']
La columna caparod tiene los valores únicos: ['Certificado cero']
La columna sexo tiene los valores únicos: ['Certificado cero']
La columna aliento tiene los valores únicos: ['Certificado cero']
La columna cinturon tiene los valores únicos: ['Certificado cero']
La columna id_edad tiene los valores únicos: [0]
La columna clasacc tiene los valores únicos: ['Certificado cero']


In [22]:
# Filtramos los que tenga en día 0.
accidentes_mexico = accidentes_mexico[accidentes_mexico.id_dia!=0]

In [23]:
accidentes_mexico.id_dia.unique()

array([11,  9, 12, 13, 10, 14, 29, 15, 30, 31, 16,  7,  5,  3,  1, 17, 18,
       22, 27, 28, 26,  2, 19, 25,  6,  4, 21, 20,  8, 23, 24, 32])

In [24]:
accidentes_mexico["id_dia"] = accidentes_mexico["id_dia"].replace({32:np.nan})

In [28]:
accidentes_mexico.diasemana.unique()

array(['Miércoles', 'Lunes', 'Jueves', 'Viernes', 'Martes', 'Sábado',
       'Domingo', 'Miercoles', 'Sabado', 'lunes', 'No especificado'],
      dtype=object)

In [29]:
accidentes_mexico["id_edad"]=accidentes_mexico["id_edad"].replace({0:np.nan,
                                                                  99:np.nan})

In [30]:
accidentes_mexico.head()

,id_entidad,id_municipio,anio,mes,id_hora,id_minuto,id_dia,diasemana,tipaccid,causaacci,caparod,sexo,aliento,cinturon,id_edad,clasacc,nom_entidad,nom_municipio
0,1,1.0,2017,1,18,30,11.0,Miércoles,Colisión con vehículo automotor,Conductor,Pavimentada,Hombre,No,Se ignora,30.0,Sólo daños,Aguascalientes,Aguascalientes
1,1,1.0,2017,1,1,50,9.0,Lunes,Colisión con motocicleta,Conductor,Pavimentada,Hombre,No,Se ignora,30.0,No fatal,Aguascalientes,Aguascalientes
2,1,1.0,2017,1,8,30,12.0,Jueves,Colisión con objeto fijo,Conductor,Pavimentada,Hombre,No,Se ignora,22.0,No fatal,Aguascalientes,Aguascalientes
3,1,1.0,2017,1,17,40,9.0,Lunes,Colisión con motocicleta,Conductor,Pavimentada,Mujer,No,Se ignora,73.0,Sólo daños,Aguascalientes,Aguascalientes
4,1,1.0,2017,1,21,10,11.0,Miércoles,Colisión con objeto fijo,Conductor,Pavimentada,Hombre,No,Se ignora,33.0,No fatal,Aguascalientes,Aguascalientes


In [32]:
accidentes_mexico.to_csv("../data/processed/atus_anual_1997_2019.csv", index=False)